# Installing the required libraries

In [2]:
%pip install -r ../requirements.txt

  Obtaining dependency information for langchain_ollama from https://files.pythonhosted.org/packages/bc/6b/486bea63bc6a9774780c72fc4a25508197584ef5cb9f983ebc12a24fa827/langchain_ollama-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for ollama<1.0.0,>=0.5.3 from https://files.pythonhosted.org/packages/be/f6/2091e50b8b6c3e6901f6eab283d5efd66fb71c86ddb1b4d68766c3eeba0f/ollama-0.5.3-py3-none-any.whl.metadata
INFO: pip is looking at multiple versions of langchain-ollama to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for langchain_ollama from https://files.pythonhosted.org/packages/b9/df/5d92427eabc2b070689de9d37ea90636ea51c1ec0ee75c362fa1995201cc/langchain_ollama-0.3.7-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importing relevant libraries

In [166]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from typing import Optional
from pydantic import BaseModel, Field
import getpass
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools.render import render_text_description # to describe tools as a string 
from langchain_core.output_parsers import JsonOutputParser # ensure JSON input for tools
from operator import itemgetter # to retrieve specific items in our chain.

# Setting up tools

In [180]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Draw
from rdkit.Chem.Draw import SimilarityMaps

from pydantic import BaseModel, Field, field_validator
from enum import Enum
from typing import Annotated

# --- Define tools ---
class MultiplyInput(BaseModel):
    a: int = Field(..., description="first integer")
    b: int = Field(..., description="second integer")

@tool("multiply", args_schema=MultiplyInput)
def multiply(a: int, b: int) -> int:
    """Multiply a with b"""
    return a * b

class AddInput(BaseModel):
    x: int = Field(..., description="first integer")
    y: int = Field(..., description="second integer")

@tool("add", args_schema=AddInput)
def add(x: int, y: int) -> int:
    """Add x and y"""
    return x + y

@tool
def converse(input: str) -> str:
    "Provide a natural language response using the user input."
    return llm.invoke(input)

# Pydantic stuff i dont like this >.<
DESC_MAP = {name: fn for name, fn in Descriptors.descList}
DESC_NAMES = sorted(DESC_MAP.keys())

class DescriptorArgs(BaseModel):
    smiles: str = Field(..., description="SMILES string of the molecule.")

    # Pylance sees 'str', but the LLM will see an enum in the schema
    descriptor: Annotated[
        str,
        Field(
            description=f"RDKit descriptor name. One of: {', '.join(DESC_NAMES[:50])} … (total {len(DESC_NAMES)})",
            json_schema_extra={"enum": DESC_NAMES},
        ),
    ]

    # Runtime validation (and optional case-insensitive normalization)
    @field_validator("descriptor")
    @classmethod
    def validate_descriptor(cls, v: str) -> str:
        v2 = v.strip()
        if v2 in DESC_MAP:
            return v2
        # case-insensitive fallback
        matches = [n for n in DESC_MAP if n.lower() == v2.lower()]
        if matches:
            return matches[0]
        raise ValueError(f"Invalid descriptor '{v}'. Choose one of: {', '.join(DESC_NAMES[:20])} …")

# TOOLS 
@tool(args_schema=DescriptorArgs)
def descriptor_calculation(smiles: str, descriptor: str) -> str:
    """Calculate molecular descriptors from a SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string"
    
    desc = {desc_name: func(mol) for desc_name, func in Descriptors.descList}
    return desc[descriptor]
    
@tool#(args_schema=DescriptorArgs)
def MolToSmiles(molecule: str) -> str:
    """Validate and canonicalize a SMILES string."""
    candidate = molecule.strip()
    mol = Chem.MolFromSmiles(candidate)
    
    if mol is None:
        raise ValueError(f"Invalid SMILES: {candidate}")
    return Chem.MolToSmiles(mol, canonical=True)


@tool
def partial_charge_calculation(smiles: str) -> str:
    """Calculate Gasteiger partial charges from a SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string"

    AllChem.Mol.ComputeGasteigerCharges(mol)
    charges = [atom.GetProp('_GasteigerCharge') for atom in mol.GetAtoms()]
    return str(charges)


@tool
def generate_png_descriptors(smiles: str) -> str:
    """Generate a 2D depiction of the molecule with descriptors annotated."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES string"
    
    AllChem.Mol.ComputeGasteigerCharges(mol)
    contribs = [mol.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mol.GetNumAtoms())]
    d2d = Draw.MolDraw2DCairo(400, 400)
    sim = SimilarityMaps.GetSimilarityMapFromWeights(mol, contribs, d2d, colorMap='jet', contourLines=10)
    sim.FinishDrawing()
    sim.WriteDrawingText('molecule.png')

    return "molecule.png"

@tool
def display_image(file_path: str) -> str:
    """Display the image from the file path."""
    from PIL import Image
    img = Image.open(file_path)
    img.show()
    return "Image displayed"



tools = [multiply, add, display_image, generate_png_descriptors, descriptor_calculation, MolToSmiles]

tools_by_name: dict[str, any] = {t.name: t for t in tools}


# setting up the LLM

In [ ]:
#config
#if not os.environ.get("GOOGLE_API_KEY"):
#  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#google_api_key = os.environ.get("GOOGLE_API_KEY")
#if not google_api_key:
#    raise ValueError("GOOGLE_API_KEY environment variable not set. Please set it in the .env file or your environment.")

def tool_chain(model_output):
    print([x for x in model_output])
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

llm = OllamaLLM(model="mistral")
query = input("Input: ")

rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | llm | JsonOutputParser() | tool_chain # pipeline the prompt into llm's template then pass the output to json parser then pass it to tool_chain

multi_system_prompt = f"""You have these tools:

{rendered_tools}

You will solve the user's task by repeatedly choosing a tool and arguments.
Return ONLY JSON with keys:
- "name": tool name to call next, or "final" to finish
- "arguments": dict of arguments for that tool (omit if name == "final")
- "final_answer": present ONLY if name == "final"

Use prior tool results (provided in the scratchpad) to decide the next step.
"""

multi_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", multi_system_prompt),
        ("user", "{input}"),
        ("assistant", "{scratchpad}"), 
    ]
)

multi_chain = multi_prompt | llm | JsonOutputParser()

def run_multi_step(user_query: str, max_steps: int = 128):
    scratchpad = ""
    for _ in range(max_steps):
        step = multi_chain.invoke({"input": user_query, "scratchpad": scratchpad})
        name = step["name"]
        print(step)
        if name == "final":
            return step.get("final_answer", "(no final answer)")
        if name not in tools_by_name:
            raise ValueError(f"Unknown tool: {name}")
        args = step.get("arguments", {}) or {}
        tool_result = tools_by_name[name].invoke(args)
        scratchpad += f"\nTOOL={name} ARGS={args} RESULT={tool_result}"
    raise RuntimeError("Max steps reached without final answer")

final_answer = run_multi_step(query)
print(query)
print(final_answer)

#chain = llm
#chain.invoke(query)

{'name': 'MolToSmiles', 'arguments': {'molecule': 'O2'}}


[22:22:59] SMILES Parse Error: unclosed ring for input: 'O2'


ValueError: Invalid SMILES: O2